In [2]:
import json
import pandas as pd
from datetime import datetime

# Charger le fichier JSON
with open("results.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []

for publication in data:
    # Nettoyage du texte de la publication
    post_text = "\n".join([p.strip() for p in publication.get("post", []) if p.strip()])
    
    # Format date (facultatif)
    raw_date = publication.get("date_publication", "").replace("Publié le ", "")
    
    # Ajouter la publication principale
    rows.append({
        "type": "publication",
        "date": raw_date,
        "texte": post_text,
        "source_url": publication.get("url", "")
    })

    # Ajouter les commentaires et replies
    for comment in publication.get("comments", []):
        comment_text = comment.get("text", "").strip()
        comment_date = comment.get("date", "").strip()
        
        rows.append({
            "type": "comment",
            "date": comment_date,
            "texte": comment_text,
            "source_url": publication.get("url", "")
        })

        for reply in comment.get("replies", []):
            reply_text = reply.get("text", "").strip()
            reply_date = reply.get("date", "").strip()
            
            rows.append({
                "type": "reply",
                "date": reply_date,
                "texte": reply_text,
                "source_url": publication.get("url", "")
            })

# Convertir en DataFrame
df = pd.DataFrame(rows)

# Sauvegarder au format CSV (ou JSON)
df.to_csv("dataset_nlp.csv", index=False, encoding="utf-8")
# df.to_json("dataset_nlp.json", orient="records", ensure_ascii=False, indent=2)


In [3]:
df

,type,date,texte,source_url
0,publication,jeudi 3 avril 2025 à 23h12min,La Coordination nationale des associations de ...,https://lefaso.net/spip.php?article137183
1,comment,4 avril 09:40,C’est la photo de famille entre donnateurs et ...,https://lefaso.net/spip.php?article137183
2,publication,mardi 1er avril 2025 à 22h30min,"1. Bien s’hydrater\nBoire au moins 1,5 à 2 lit...",https://lefaso.net/spip.php?article137125
3,publication,samedi 29 mars 2025 à 21h51min,"François Sama a soutenu, ce vendredi 28 mars 2...",https://lefaso.net/spip.php?article137081
4,comment,30 mars 09:30,"Une thèse de philosophie sur Spinoza pour ""con...",https://lefaso.net/spip.php?article137081
...,...,...,...,...
5016,comment,24 décembre 2024 09:16,"Suite à ce discours extrêmement déterminé, on ...",https://lefaso.net/spip.php?article135011
5017,comment,25 décembre 2024 00:54,Tous les discours tendant à disculper la Franc...,https://lefaso.net/spip.php?article135011
5018,reply,25 décembre 2024 21:41,Difficile de comprendre ton idée car va dans t...,https://lefaso.net/spip.php?article135011
5019,comment,27 décembre 2024 08:19,"@EcowasBoy, vous n’êtes et vous ne serez jamai...",https://lefaso.net/spip.php?article135011


In [ ]:
# Afficher avec une mise en forme (indentation)
print(json.dumps(data, indent=4, ensure_ascii=False))